#Package Section

In [25]:
import sys
import numpy as np
import copy
from numpy import linalg as LA
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
import time
# for sparse matrix
from scipy import sparse
#early stop
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint


#Classes and functions

In [26]:
# invalide devide resutls will be nan
np.seterr(divide='ignore', invalid='ignore')

############------------graph_encoder_embed_start----------------###############
class GraphEncoderEmbed:
  def run(self, X, Y, n, **kwargs):
    defaultKwargs = {'EdgeList': False, 'DiagA': True, 'Laplacian': False, 'Correlation': True}
    kwargs = { **defaultKwargs, **kwargs}

    if kwargs['EdgeList']:
      size_flag = self.edge_list_size
      X = self.Edge_to_Sparce(X, n, size_flag)
    
    if kwargs['DiagA']:
      X = self.Diagnal(X, n)

    if kwargs['Laplacian']:
      X = self.Laplacian(X, n)
    
    Z, W = self.Basic(X, Y, n)

    if kwargs['Correlation']:
      Z = self.Correlation(Z)
    
    return Z, W

  def Basic(self, X, Y, n):
    """
      graph embedding basic function
      input X is sparse csr matrix of adjacency matrix
      -- if there is a connection between node i and node j:
      ---- X(i,j) = 1, no edge weight
      ---- X(i,j) = edge weight.
      -- if there is no connection between node i and node j:
      ---- X(i,j) = 0, 
      ---- note there is no storage for this in sparse matrix. 
      ---- No storage means 0 in sparse matrix.
      input Y is numpy array with size (n,1):
      -- value -1 indicate no lable
      -- value >=0 indicate real label
      input train_idx: a list of indices of input X for training set 
    """
    # assign k to the max along the first column
    # Note for python, label Y starts from 0. Python index starts from 0. thus size k should be max + 1
    k = Y[:,0].max() + 1
    
    #nk: 1*n array, contains the number of observations in each class
    nk = np.zeros((1,k))
    for i in range(k):
      nk[0,i] = np.count_nonzero(Y[:,0]==i)
    
    #W: sparse matrix for encoder marix. W[i,k] = {1/nk if Yi==k, otherwise 0}
    W = sparse.dok_matrix((n, k), dtype=np.float32)

    for i in range(n):
      k_i = Y[i,0]
      if k_i >=0:
        W[i,k_i] = 1/nk[0,k_i]
    
    W = sparse.csr_matrix(W)
    Z = X.dot(W)
    
    return Z, W

  def Diagnal(self, X, n):
    """
      input X is sparse csr matrix of adjacency matrix
      return a sparce csr matrix of X matrix with 1s on the diagnal
    """
    I = sparse.identity(n)
    X = X + I
    return X


  def Laplacian(self, X, n):
    """
      input X is sparse csr matrix of adjacency matrix
      return a sparce csr matrix of Laplacian normalization of X matrix
    """
    X_sparse = sparse.csr_matrix(X)
    # get an array of degrees
    dig = X_sparse.sum(axis=0).A1
    # diagnal sparce matrix of D
    D = sparse.diags(dig,0)
    _D = D.power(-0.5)
    # D^-0.5 x A x D^-0.5
    L = _D.dot(X_sparse.dot(_D)) 

    # _L = _D.dot(X_sparse.dot(_D))    
    # # L = I - D^-0.5 x A x D^-0.5
    # I = sparse.identity(n)
    # L = I - _L   

    return L
  
  def Correlation(self, Z):
    """
      input Z is sparse csr matrix of embedding matrix from the basic function
      return normalized Z sparce matrix
      Calculation:
      Calculate each row's 2-norm (Euclidean distance). 
      e.g.row_x: [ele_i,ele_j,ele_k]. norm2 = sqr(sum(ele_i^2+ele_i^2+ele_i^2))
      then divide each element by their row norm
      e.g. [ele_i/norm2,ele_j/norm2,ele_k/norm2] 
    """
    # 2-norm
    row_norm = sparse.linalg.norm(Z, axis = 1)

    # row division to get the normalized Z
    diag = np.nan_to_num(1/row_norm)
    N = sparse.diags(diag,0)
    Z = N.dot(Z)

    return Z

  def edge_list_size(self, X):
    """
      set default edge list size as S3.
      If find X only has 2 columns, 
      return a flag "S2" indicating this is S2 edge list
    """
    if X.shape[1] == 2:
      return "S2"
    else:
      return "S3"
    
  def Edge_to_Sparce(self, X, n, size_flag):
    """
      input X is an edge list.
      For S2 edge list (e.g. node_i, node_j per row), add one to all connections
      return a sparce csr matrix of S3 edge list
    """   
    #Build an empty sparse matrix. 
    X_new = sparse.dok_matrix((n, n), dtype=np.float32)

    for row in X:
      if size_flag == "S2":
        [node_i, node_j] = row
        X_new[node_i, node_j] = 1
      else:
        [node_i, node_j, weight] = row
        X_new[node_i, node_j] = weight
    
    X_new = sparse.csr_matrix(X_new)

    return X_new


############------------graph_encoder_embed_end------------------###############
############------------Sparse_supervised_learning_start---------###############

# https://www.kaggle.com/c/talkingdata-mobile-user-demographics/discussion/22567
# https://github.com/tkipf/pygcn/blob/1600b5b748b3976413d1e307540ccc62605b4d6d/pygcn/utils.py#L73

def batch_generator(X, y, k, batch_size, shuffle):
    number_of_batches = X.shape[0]/batch_size
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = to_categorical(y[batch_index], num_classes=k)
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

class Hyperperameters:
  """
    define perameters for GNN.
    default values are for GNN learning -- "Leaner" ==2:
      embedding via partial label, then learn unknown label via two-layer NN

  """
  def __init__(self):
    # there is no scaled conjugate gradiant in keras optimiser, use defualt instead
    # use whatever default
    self.learning_rate = 0.01  # Initial learning rate.
    self.epochs = 100 #Number of epochs to train.
    self.hidden = 20 #Number of units in hidden layer 
    self.val_split = 0.1 #Split 10% of training data for validation
    self.loss = 'categorical_crossentropy' # loss function

class GNN:
  def __init__(self, DataSets):
    GNN.DataSets = DataSets
    GNN.hyperM = Hyperperameters()
    GNN.model = self.GNN_model()  #model summary: GNN.model.summary()
      
 
  def GNN_model(self):
    """
      build GNN model
    """
    hyperM = self.hyperM
    DataSets = self.DataSets

    z_train = DataSets.z_train
    k = DataSets.d

    feature_num = z_train.shape[1]
    
    model = keras.Sequential([
    keras.layers.Flatten(input_shape = (feature_num,)),  # input layer 
    keras.layers.Dense(hyperM.hidden, activation='relu'),  # hidden layer -- no tansig activation function in Keras, use relu instead
    keras.layers.Dense(k, activation='softmax') # output layer, matlab used softmax for patternnet default ??? max(opts.neuron,K)? opts 
    ])

    optimizer = keras.optimizers.Adam(learning_rate = hyperM.learning_rate)

    model.compile(optimizer='adam',
                  loss=hyperM.loss,
                  metrics=['accuracy'])

    return model
    
  def GNN_run(self):
    """
      Train and test directly.
      Do not learn from the unknown labels.
    """
    gnn = copy.deepcopy(self)
    hyperM = gnn.hyperM
    DataSets = self.DataSets
    k = DataSets.d
    z_train = DataSets.z_train
    y_train = DataSets.y_train
    y_test = DataSets.y_test
    z_test = DataSets.z_test
    model = gnn.model    

    early_stopping_callback = EarlyStopping(monitor='loss', patience=5, verbose=0)
    checkpoint_callback = ModelCheckpoint('GNN.h5', monitor='loss', save_best_only=True, mode='min', verbose=0)
    
    train_strat = time.time()
    history = model.fit(batch_generator(z_train, y_train, k, 32, True),
                    epochs=hyperM.epochs,
                    steps_per_epoch=z_train.shape[0],
                    callbacks=[early_stopping_callback, checkpoint_callback],
                    verbose=0)
    train_end = time.time()
    train_time = train_end - train_strat 

    y_test_one_hot = to_categorical(y_test) 
    # set verbose to 0 to silent the output
    test_loss, test_acc = gnn.model.evaluate(z_test.toarray(),  y_test_one_hot, verbose=0) 
    return test_acc, train_time
############------------Sparse_supervised_learning_end---------###############


#Packages for Drive Files

In [18]:
# import packages
## for mount drive purpose
import os
from google.colab import drive

#Mount Drive

In [19]:
# mount drive
drive.mount('/content/drive/', force_remount=True)
os.chdir('/content/drive/My Drive/Colab_Notebooks/Graph_ML/semi_dr.shen')

Mounted at /content/drive/


# import ipynb packages

In [20]:
!pip install import-ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import import_ipynb
from test_cases import Model, Case

# Test Cases 

## Test sparse matrix

In [ ]:
A = sparse.csr_matrix(np.array([[4,5,6], [7,8,9],[10,11,12]]))
B = sparse.csr_matrix(np.array([[1,0,0], [0,2,0],[0,0,3]]))

In [ ]:
print(B)

  (0, 0)	1
  (1, 1)	2
  (2, 2)	3


In [ ]:
print(A)

  (0, 0)	4
  (0, 1)	5
  (0, 2)	6
  (1, 0)	7
  (1, 1)	8
  (1, 2)	9
  (2, 0)	10
  (2, 1)	11
  (2, 2)	12


In [ ]:
print(A.nonzero())

(array([0, 0, 0, 1, 1, 1, 2, 2, 2], dtype=int32), array([0, 1, 2, 0, 1, 2, 0, 1, 2], dtype=int32))


In [ ]:
print(B.nonzero())

(array([0, 1, 2], dtype=int32), array([0, 1, 2], dtype=int32))


In [ ]:
C = A.dot(B)

In [ ]:
print(C)

  (0, 2)	18
  (0, 1)	10
  (0, 0)	4
  (1, 2)	27
  (1, 1)	16
  (1, 0)	7
  (2, 2)	36
  (2, 1)	22
  (2, 0)	10


In [ ]:
# this is csr, using csc will be faster to slice columns
C_new = sparse.lil_matrix(C[:,[0,2]])

In [ ]:
print(C_new)

  (0, 0)	4
  (0, 1)	18
  (1, 0)	7
  (1, 1)	27
  (2, 0)	10
  (2, 1)	36


### Test diagnal, laplacian

In [ ]:
A = np.array([
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 1],
 [0, 1, 1, 0]])
print(A)

[[0 0 1 0]
 [0 0 0 1]
 [1 0 0 1]
 [0 1 1 0]]


In [ ]:
I = sparse.identity(4)

In [ ]:
A = A + I
print(A)

[[1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [1. 0. 1. 1.]
 [0. 1. 1. 1.]]


In [ ]:
import networkx as nx

In [ ]:
G=nx.from_numpy_matrix(A)
G.degree()

DegreeView({0: 3, 1: 3, 2: 4, 3: 4})

In [ ]:
A_sparse = sparse.csr_matrix(A)

In [ ]:
# array of degrees
dig = A_sparse.sum(axis=0).A1

In [ ]:
print(dig)

[1 1 2 2]


In [ ]:
from scipy.sparse import diags

In [ ]:
D = diags(dig,0)
_D = D.power(-0.5)
_L = _D.dot(A_sparse.dot(_D))
I = sparse.identity(4)
L = I - _L

In [ ]:
print(D)

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	2.0
  (3, 3)	2.0


In [ ]:
print(_L)

  (0, 2)	0.7071067811865476
  (1, 3)	0.7071067811865476
  (2, 0)	0.7071067811865476
  (2, 3)	0.5000000000000001
  (3, 1)	0.7071067811865476
  (3, 2)	0.5000000000000001


In [ ]:
I = sparse.identity(4)
L = I - _L
print(L.toarray())

[[ 1.          0.         -0.70710678  0.        ]
 [ 0.          1.          0.         -0.70710678]
 [-0.70710678  0.          1.         -0.5       ]
 [ 0.         -0.70710678 -0.5         1.        ]]


In [ ]:
G=nx.from_numpy_matrix(A)
L_matrix = nx.normalized_laplacian_matrix(G).toarray() 
print(L_matrix)

[[ 1.          0.         -0.70710678  0.        ]
 [ 0.          1.          0.         -0.70710678]
 [-0.70710678  0.          1.         -0.5       ]
 [ 0.         -0.70710678 -0.5         1.        ]]


## Graph Encoder test case


In [ ]:
class Encoder_case:
  def __init__(self, A,Y,n):
    Encoder_case.X = A
    Encoder_case.Y = Y
    Encoder_case.n = n

###Case 1

A = 

\begin{bmatrix}
0 & 1 & 1 & 1 & 0\\
1 & 0 & 1 & 1 & 1\\
1 & 1 & 0 & 1 & 1\\
1 & 1 & 1 & 0 & 1\\
0 & 1 & 1 & 1 & 0
\end{bmatrix}

Labels = [0,0,0,1,1] 


In [ ]:
A = np.ones((5,5))
A[0,4] = 0
A[4,0] = 0
np.fill_diagonal(A, 0)

Y = np.array([[0,0,0,1,1]]).reshape((5,1))

print(A)
print(Y)

Encoder_case = Encoder_case(A,Y,5)

[[0. 1. 1. 1. 0.]
 [1. 0. 1. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 0. 1.]
 [0. 1. 1. 1. 0.]]
[[0]
 [0]
 [0]
 [1]
 [1]]


In [ ]:
print(np.nonzero(A))

(array([0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4]), array([1, 2, 3, 0, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 4, 1, 2, 3]))


In [ ]:
np.transpose(np.nonzero(A))

array([[0, 1],
       [0, 2],
       [0, 3],
       [1, 0],
       [1, 2],
       [1, 3],
       [1, 4],
       [2, 0],
       [2, 1],
       [2, 3],
       [2, 4],
       [3, 0],
       [3, 1],
       [3, 2],
       [3, 4],
       [4, 1],
       [4, 2],
       [4, 3]])

In [ ]:
len(np.transpose(np.nonzero(A)))

18

#### Test sparse correlation

In [ ]:
GEE = GraphEncoderEmbed()
X_sparse = sparse.csr_matrix(Encoder_case.X)
Z, W = GEE.run(X_sparse, Encoder_case.Y, Encoder_case.n, Laplacian = False, DiagA = False, Correlation = False)

In [ ]:
# 2-norm
row_norm = sparse.linalg.norm(Z, axis = 1)
print(row_norm)

[0.83333335 1.20185044 1.20185044 1.11803402 0.83333335]


In [ ]:
# get 1/row_norm first
diag = np.nan_to_num(1/row_norm)
print(diag)

[1.19999998 0.83205029 0.83205029 0.89442717 1.19999998]


In [ ]:
# make to sparce diagnal
N = sparse.diags(diag,0)
print(N)

  (0, 0)	1.1999999771118166
  (1, 1)	0.8320502867079879
  (2, 2)	0.8320502867079879
  (3, 3)	0.8944271696751103
  (4, 4)	1.1999999771118166


In [ ]:
# Z * diagnal (1/row_norm)
Z_r = N.dot(Z)
print(Z_r)

  (0, 1)	0.5999999885559083
  (0, 0)	0.8000000085830685
  (1, 1)	0.8320502867079879
  (1, 0)	0.5547002076700125
  (2, 1)	0.8320502867079879
  (2, 0)	0.5547002076700125
  (3, 1)	0.44721358483755513
  (3, 0)	0.8944271963311171
  (4, 1)	0.5999999885559083
  (4, 0)	0.8000000085830685


#### [Original]Laplacian = False, correclation = False, DiagA = False

In [ ]:
Dataset = DataPreprocess(Encoder_case, Laplacian = False, DiagA = False)
print(Dataset.X)
print(Dataset.Y)
print(Dataset.n)

[array([[0., 1., 1.],
       [0., 2., 1.],
       [0., 3., 1.],
       [1., 0., 1.],
       [1., 2., 1.],
       [1., 3., 1.],
       [1., 4., 1.],
       [2., 0., 1.],
       [2., 1., 1.],
       [2., 3., 1.],
       [2., 4., 1.],
       [3., 0., 1.],
       [3., 1., 1.],
       [3., 2., 1.],
       [3., 4., 1.],
       [4., 1., 1.],
       [4., 2., 1.],
       [4., 3., 1.]])]
[[0]
 [0]
 [0]
 [1]
 [1]]
5


In [ ]:
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = False)
print(Z)
print(W)

[[1.33333333 1.        ]
 [1.33333333 2.        ]
 [1.33333333 2.        ]
 [2.         1.        ]
 [1.33333333 1.        ]]
[[0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.         0.5       ]
 [0.         0.5       ]]


#### SPARSE MATRIX, Laplacian = False, correclation = False, DiagA = False
note: z matrix valus is 1/2 of the edge list version. doesn't affect the training and testing does not adding double weights here.

In [ ]:
GEE = GraphEncoderEmbed()
X_sparse = sparse.csr_matrix(Encoder_case.X)
Z, W = GEE.run(X_sparse, Encoder_case.Y, Encoder_case.n, Laplacian = False, DiagA = False, Correlation = False)

In [ ]:
print(Z)
print(W)

  (0, 1)	0.5
  (0, 0)	0.6666666865348816
  (1, 1)	1.0
  (1, 0)	0.6666666865348816
  (2, 1)	1.0
  (2, 0)	0.6666666865348816
  (3, 1)	0.5
  (3, 0)	1.0000000298023224
  (4, 1)	0.5
  (4, 0)	0.6666666865348816
  (0, 0)	0.33333334
  (1, 0)	0.33333334
  (2, 0)	0.33333334
  (3, 1)	0.5
  (4, 1)	0.5


In [ ]:
print(Z.toarray())
print(W.toarray())

[[0.66666669 0.5       ]
 [0.66666669 1.        ]
 [0.66666669 1.        ]
 [1.00000003 0.5       ]
 [0.66666669 0.5       ]]
[[0.33333334 0.        ]
 [0.33333334 0.        ]
 [0.33333334 0.        ]
 [0.         0.5       ]
 [0.         0.5       ]]


#### Laplacian = False, correclation = True, DiagA = False

In [ ]:
Dataset = DataPreprocess(Encoder_case, Laplacian = False, DiagA = False)
print(Dataset.X)
print(Dataset.Y)
print(Dataset.n)

[array([[0., 1., 1.],
       [0., 2., 1.],
       [0., 3., 1.],
       [1., 0., 1.],
       [1., 2., 1.],
       [1., 3., 1.],
       [1., 4., 1.],
       [2., 0., 1.],
       [2., 1., 1.],
       [2., 3., 1.],
       [2., 4., 1.],
       [3., 0., 1.],
       [3., 1., 1.],
       [3., 2., 1.],
       [3., 4., 1.],
       [4., 1., 1.],
       [4., 2., 1.],
       [4., 3., 1.]])]
[[0]
 [0]
 [0]
 [1]
 [1]]
5


In [ ]:
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = True)
print(Z)
print(W)

[[0.8        0.6       ]
 [0.5547002  0.83205029]
 [0.5547002  0.83205029]
 [0.89442719 0.4472136 ]
 [0.8        0.6       ]]
[[0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.         0.5       ]
 [0.         0.5       ]]


#### SPARSE MATRIX, Laplacian = False, correclation = True, DiagA = False
note: z matrix valus is 1/2 of the edge list version. doesn't affect the training and testing does not adding double weights here.

In [ ]:
GEE = GraphEncoderEmbed()
X_sparse = sparse.csr_matrix(Encoder_case.X)
Z, W = GEE.run(X_sparse, Encoder_case.Y, Encoder_case.n, Laplacian = False, DiagA = False, Correlation = True)

In [ ]:
print(Z)
print(W)

  (0, 1)	0.5999999885559083
  (0, 0)	0.8000000085830685
  (1, 1)	0.8320502867079879
  (1, 0)	0.5547002076700125
  (2, 1)	0.8320502867079879
  (2, 0)	0.5547002076700125
  (3, 1)	0.44721358483755513
  (3, 0)	0.8944271963311171
  (4, 1)	0.5999999885559083
  (4, 0)	0.8000000085830685
  (0, 0)	0.33333334
  (1, 0)	0.33333334
  (2, 0)	0.33333334
  (3, 1)	0.5
  (4, 1)	0.5


In [ ]:
print(Z.toarray())
print(W.toarray())

[[0.80000001 0.59999999]
 [0.55470021 0.83205029]
 [0.55470021 0.83205029]
 [0.8944272  0.44721358]
 [0.80000001 0.59999999]]
[[0.33333334 0.        ]
 [0.33333334 0.        ]
 [0.33333334 0.        ]
 [0.         0.5       ]
 [0.         0.5       ]]


#### Laplacian = True, correclation = False, DiagA = False

In [ ]:
Dataset = DataPreprocess(Encoder_case, Laplacian = True, DiagA = False)
print(Dataset.X)
print(Dataset.Y)
print(Dataset.n)

[array([[0.        , 1.        , 0.14433757],
       [0.        , 2.        , 0.14433757],
       [0.        , 3.        , 0.14433757],
       [1.        , 0.        , 0.14433757],
       [1.        , 2.        , 0.125     ],
       [1.        , 3.        , 0.125     ],
       [1.        , 4.        , 0.14433757],
       [2.        , 0.        , 0.14433757],
       [2.        , 1.        , 0.125     ],
       [2.        , 3.        , 0.125     ],
       [2.        , 4.        , 0.14433757],
       [3.        , 0.        , 0.14433757],
       [3.        , 1.        , 0.125     ],
       [3.        , 2.        , 0.125     ],
       [3.        , 4.        , 0.14433757],
       [4.        , 1.        , 0.14433757],
       [4.        , 2.        , 0.14433757],
       [4.        , 3.        , 0.14433757]])]
[[0]
 [0]
 [0]
 [1]
 [1]]
5


In [ ]:
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = False)
print(Z)
print(W)

[[0.19245009 0.14433757]
 [0.17955838 0.26933757]
 [0.17955838 0.26933757]
 [0.26289171 0.14433757]
 [0.19245009 0.14433757]]
[[0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.         0.5       ]
 [0.         0.5       ]]


#### SPARSE MATRIX, Laplacian = True, correclation = False, DiagA = False
note: z matrix valus is 1/2 of the edge list version. doesn't affect the training and testing does not adding double weights here.

In [ ]:
GEE = GraphEncoderEmbed()
X_sparse = sparse.csr_matrix(Encoder_case.X)
Z, W = GEE.run(X_sparse, Encoder_case.Y, Encoder_case.n, Laplacian = True, DiagA = False, Correlation = False)

In [ ]:
print(Z)
print(W)

  (0, 1)	0.14433756729740643
  (0, 0)	0.19245009546533487
  (1, 1)	0.26933756729740643
  (1, 0)	0.17955838354952763
  (2, 1)	0.26933756729740643
  (2, 0)	0.17955838354952763
  (3, 1)	0.14433756729740643
  (3, 0)	0.26289171936638783
  (4, 1)	0.14433756729740643
  (4, 0)	0.19245009546533487
  (0, 0)	0.33333334
  (1, 0)	0.33333334
  (2, 0)	0.33333334
  (3, 1)	0.5
  (4, 1)	0.5


In [ ]:
print(Z.toarray())
print(W.toarray())

[[0.1924501  0.14433757]
 [0.17955838 0.26933757]
 [0.17955838 0.26933757]
 [0.26289172 0.14433757]
 [0.1924501  0.14433757]]
[[0.33333334 0.        ]
 [0.33333334 0.        ]
 [0.33333334 0.        ]
 [0.         0.5       ]
 [0.         0.5       ]]


## [Sparse]Supervised Learning

### Case 10 with 3000 nodes (SBM)

In [ ]:
n = 3000
case = Case(n)

In [ ]:
case_10 = case.case_10_fully_known()
case_10.summary()

name:

    SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
Y:
(3000, 1)
[[1]
 [0]
 [2]
 ...
 [2]
 [2]
 [2]]


#### Laplacian = False, DiagA = False, Correlation = False

In [ ]:
X_sparse = sparse.csr_matrix(case_10.X)
GEE = GraphEncoderEmbed()
emb_strat = time.time()
Z, W = GEE.run(X_sparse, case_10.Y, case_10.n, Laplacian = False, DiagA = False, Correlation = False)
emb_end = time.time()
emb_time = emb_end - emb_strat

train_idx = case_10.train_idx
test_idx = case_10.test_idx
case_10.z_train= Z[train_idx]
case_10.z_test = Z[test_idx]
case_10.y_train = case_10.Y[train_idx].ravel()
case_10.y_test = case_10.Y_test.ravel() 

gnn = GNN(case_10)
acc, train_time  = gnn.GNN_run()

print("--- embed %s seconds ---" % emb_time)
print("--- train %s seconds ---" % train_time)
print("--- accuracy: %s ---" % acc)

--- embed 0.05303168296813965 seconds ---
--- train 161.20756196975708 seconds ---
--- accuracy: 0.9566666483879089 ---


#### Laplacian = False, DiagA = True, Correlation = False

In [ ]:
X_sparse = sparse.csr_matrix(case_10.X)
GEE = GraphEncoderEmbed()
emb_strat = time.time()
Z, W = GEE.run(X_sparse, case_10.Y, case_10.n, Laplacian = False, DiagA = True, Correlation = False)
emb_end = time.time()
emb_time = emb_end - emb_strat

train_idx = case_10.train_idx
test_idx = case_10.test_idx
case_10.z_train= Z[train_idx]
case_10.z_test = Z[test_idx]
case_10.y_train = case_10.Y[train_idx].ravel()
case_10.y_test = case_10.Y_test.ravel() 

gnn = GNN(case_10)
acc, train_time  = gnn.GNN_run()

print("--- embed %s seconds ---" % emb_time)
print("--- train %s seconds ---" % train_time)
print("--- accuracy: %s ---" % acc)

--- embed 0.053815603256225586 seconds ---
--- train 205.69926357269287 seconds ---
--- accuracy: 0.9566666483879089 ---


#### Laplacian = True, DiagA = False, Correlation = False

In [ ]:
X_sparse = sparse.csr_matrix(case_10.X)
GEE = GraphEncoderEmbed()
emb_strat = time.time()
Z, W = GEE.run(X_sparse, case_10.Y, case_10.n, Laplacian = True, DiagA = False, Correlation = False)
emb_end = time.time()
emb_time = emb_end - emb_strat

train_idx = case_10.train_idx
test_idx = case_10.test_idx
case_10.z_train= Z[train_idx]
case_10.z_test = Z[test_idx]
case_10.y_train = case_10.Y[train_idx].ravel()
case_10.y_test = case_10.Y_test.ravel() 

gnn = GNN(case_10)
acc, train_time  = gnn.GNN_run()

print("--- embed %s seconds ---" % emb_time)
print("--- train %s seconds ---" % train_time)
print("--- accuracy: %s ---" % acc)

--- embed 0.07335996627807617 seconds ---
--- train 460.2924311161041 seconds ---
--- accuracy: 0.95333331823349 ---


### Case 20 with 5000 nodes (DC-SBM)

In [31]:
n = 5000
case = Case(n)

In [32]:
case_20 = case.case_20_fully_known()
case_20.summary()

[[0.20530728]
 [0.25104189]
 [0.22868416]
 ...
 [0.06156643]
 [0.27507487]
 [0.12602176]]
name:

    DC-SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
5000
d:
<class 'int'>
3
X:
(5000, 5000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(5000, 1)
[[1]
 [0]
 [2]
 ...
 [2]
 [0]
 [1]]


In [33]:
X_sparse = sparse.csr_matrix(case_20.X)
GEE = GraphEncoderEmbed()
emb_strat = time.time()
Z, W = GEE.run(X_sparse, case_20.Y, case_20.n, Laplacian = False, DiagA = False, Correlation = False)
emb_end = time.time()
emb_time = emb_end - emb_strat

train_idx = case_20.train_idx
test_idx = case_20.test_idx
case_20.z_train= Z[train_idx]
case_20.z_test = Z[test_idx]
case_20.y_train = case_20.Y[train_idx].ravel()
case_20.y_test = case_20.Y_test.ravel() 

gnn = GNN(case_20)
acc, train_time  = gnn.GNN_run()

print("--- embed %s seconds ---" % emb_time)
print("--- train %s seconds ---" % train_time)
print("--- accuracy: %s ---" % acc)

--- embed 0.054897308349609375 seconds ---
--- train 258.6013958454132 seconds ---
--- accuracy: 0.9350000023841858 ---


In [ ]:
X_sparse = sparse.csr_matrix(case_20.X)
GEE = GraphEncoderEmbed()
emb_strat = time.time()
Z, W = GEE.run(X_sparse, case_20.Y, case_20.n, Laplacian = False, DiagA = True, Correlation = False)
emb_end = time.time()
emb_time = emb_end - emb_strat

train_idx = case_20.train_idx
test_idx = case_20.test_idx
case_20.z_train= Z[train_idx]
case_20.z_test = Z[test_idx]
case_20.y_train = case_20.Y[train_idx].ravel()
case_20.y_test = case_20.Y_test.ravel() 

gnn = GNN(case_20)
acc, train_time  = gnn.GNN_run()

print("--- embed %s seconds ---" % emb_time)
print("--- train %s seconds ---" % train_time)
print("--- accuracy: %s ---" % acc)

--- embed 0.059488773345947266 seconds ---
--- train 507.97558784484863 seconds ---
--- accuracy: 0.9369999766349792 ---


In [ ]:
X_sparse = sparse.csr_matrix(case_20.X)
GEE = GraphEncoderEmbed()
emb_strat = time.time()
Z, W = GEE.run(X_sparse, case_20.Y, case_20.n, Laplacian = False, DiagA = False, Correlation = True)
emb_end = time.time()
emb_time = emb_end - emb_strat

train_idx = case_20.train_idx
test_idx = case_20.test_idx
case_20.z_train= Z[train_idx]
case_20.z_test = Z[test_idx]
case_20.y_train = case_20.Y[train_idx].ravel()
case_20.y_test = case_20.Y_test.ravel() 

gnn = GNN(case_20)
acc, train_time  = gnn.GNN_run()

print("--- embed %s seconds ---" % emb_time)
print("--- train %s seconds ---" % train_time)
print("--- accuracy: %s ---" % acc)

--- embed 0.05906963348388672 seconds ---
--- train 651.0715844631195 seconds ---
--- accuracy: 0.9350000023841858 ---


### Case 21 with 3000 nodes (DC-SBM Edge List)

In [28]:
n = 3000
case = Case(n)

In [29]:
case_21 = case.case_21_fully_known()
case_21.summary()

[[0.08059155]
 [0.35099271]
 [0.11881569]
 ...
 [0.03571626]
 [0.31130134]
 [0.18023799]]
name:

    DC-SBM with 10 classes and defined probabilities with fully known labels.
    Edge list version. 
    
n:
<class 'int'>
3000
d:
<class 'int'>
10
X:
(60974, 3)
[[   0    3    1]
 [   0  168    1]
 [   0  551    1]
 ...
 [2999 2577    1]
 [2999 2877    1]
 [2999 2951    1]]
Y:
(3000, 1)
[[4]
 [0]
 [5]
 ...
 [6]
 [7]
 [5]]


In [30]:
GEE = GraphEncoderEmbed()
emb_strat = time.time()
Z, W = GEE.run(case_21.X, case_21.Y, case_21.n, Laplacian = False, DiagA = False, Correlation = False, EdgeList = True)
emb_end = time.time()
emb_time = emb_end - emb_strat

train_idx = case_21.train_idx
test_idx = case_21.test_idx
case_21.z_train= Z[train_idx]
case_21.z_test = Z[test_idx]
case_21.y_train = case_21.Y[train_idx].ravel()
case_21.y_test = case_21.Y_test.ravel() 

gnn = GNN(case_21)
acc, train_time  = gnn.GNN_run()

print("--- embed %s seconds ---" % emb_time)
print("--- train %s seconds ---" % train_time)
print("--- accuracy: %s ---" % acc)


--- embed 0.8013429641723633 seconds ---
--- train 217.37675404548645 seconds ---
--- accuracy: 0.824999988079071 ---


In [ ]:
GEE = GraphEncoderEmbed()
emb_strat = time.time()
Z, W = GEE.run(case_21.X, case_21.Y, case_21.n, Laplacian = True, DiagA = False, Correlation = False, EdgeList = True)
emb_end = time.time()
emb_time = emb_end - emb_strat

train_idx = case_21.train_idx
test_idx = case_21.test_idx
case_21.z_train= Z[train_idx]
case_21.z_test = Z[test_idx]
case_21.y_train = case_21.Y[train_idx].ravel()
case_21.y_test = case_21.Y_test.ravel() 

gnn = GNN(case_21)
acc, train_time  = gnn.GNN_run()

print("--- embed %s seconds ---" % emb_time)
print("--- train %s seconds ---" % train_time)
print("--- accuracy: %s ---" % acc)


--- embed 0.8346648216247559 seconds ---
--- train 481.73461079597473 seconds ---
--- accuracy: 0.8183333277702332 ---


In [ ]:
GEE = GraphEncoderEmbed()
emb_strat = time.time()
Z, W = GEE.run(case_21.X, case_21.Y, case_21.n, Laplacian = False, DiagA = True, Correlation = False, EdgeList = True)
emb_end = time.time()
emb_time = emb_end - emb_strat

train_idx = case_21.train_idx
test_idx = case_21.test_idx
case_21.z_train= Z[train_idx]
case_21.z_test = Z[test_idx]
case_21.y_train = case_21.Y[train_idx].ravel()
case_21.y_test = case_21.Y_test.ravel() 

gnn = GNN(case_21)
acc, train_time  = gnn.GNN_run()

print("--- embed %s seconds ---" % emb_time)
print("--- train %s seconds ---" % train_time)
print("--- accuracy: %s ---" % acc)

--- embed 0.7735185623168945 seconds ---
--- train 502.3515696525574 seconds ---
--- accuracy: 0.8199999928474426 ---


In [ ]:
GEE = GraphEncoderEmbed()
emb_strat = time.time()
Z, W = GEE.run(case_21.X, case_21.Y, case_21.n, Laplacian = False, DiagA = False, Correlation = True, EdgeList = True)
emb_end = time.time()
emb_time = emb_end - emb_strat

train_idx = case_21.train_idx
test_idx = case_21.test_idx
case_21.z_train= Z[train_idx]
case_21.z_test = Z[test_idx]
case_21.y_train = case_21.Y[train_idx].ravel()
case_21.y_test = case_21.Y_test.ravel() 

gnn = GNN(case_21)
acc, train_time  = gnn.GNN_run()

print("--- embed %s seconds ---" % emb_time)
print("--- train %s seconds ---" % train_time)
print("--- accuracy: %s ---" % acc)

--- embed 0.7747249603271484 seconds ---
--- train 473.5367114543915 seconds ---
--- accuracy: 0.824999988079071 ---


## [Original]Supervised Learning

In [ ]:
n = 3000
case = Case(n)

In [ ]:
# get all combinations of different emb settings 

sets_no = 8
L_set = [True, False]
Diag_set = [True, False]
Corre_set = [True, False]
comb = [L_set, Diag_set, Corre_set]
comb_set = []

ele_list = [None, None, None]
for ele1 in comb[0]:
  ele_list[0] = ele1
  for ele2 in comb[1]:
    ele_list[1] = ele2
    for ele3 in comb[2]:
      ele_list[2] = ele3
      comb_set.append(ele_list.copy())

print(comb_set)
print(len(comb_set))

[[True, True, True], [True, True, False], [True, False, True], [True, False, False], [False, True, True], [False, True, False], [False, False, True], [False, False, False]]
8


In [ ]:
def average_restuls(case_num, comb_set, learner_no):
  results = []
  for comb in comb_set:
    acc_final, train_time_final, emb_time_final, total_time_final = 0,0,0,0
    for i in range(10):
      test_case = copy.deepcopy(case_num)
      acc, train_time, emb_time, total_time = Run(test_case, "su", Learner = learner_no, Laplacian = comb[0], DiagA = comb[1], Correlation = comb[2])
      acc_final += acc
      train_time_final += train_time
      emb_time_final += emb_time 
      total_time_final += total_time
    
    acc_final /= 10
    train_time_final /= 10
    emb_time_final /= 10
    total_time_final /= 10

    result = comb + [acc_final, train_time_final, emb_time_final, total_time_final]
    results.append(result)
  
  return results

def plot(results):
  df = pd.DataFrame(results,
  index=['set_01','set_02','set_03','set_04','set_05','set_06','set_07','set_08'],
  columns=['Laplacian','DiagA', 'Correlation', 'Accuracy', 'Train_Time(s)', 'Emb_Time(s)', 'Total_Time(s)'])

  df = df.style.format({
    'Emb_Time(s)': '{:0.2f}',
    'Train_Time(s)': '{:0.5f}',
    'Total_Time(s)': '{:0.2f}'
  })

  display(df)

### Supervised

#### GNN

##### case 10

In [ ]:
case_10 = case.case_10_fully_known()
case_10.summary()

name:

    SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
Y:
(3000, 1)
[[1]
 [0]
 [2]
 ...
 [2]
 [2]
 [2]]


In [ ]:
print(case_10.bd)

0.13


In [ ]:
results = average_restuls(case_10, comb_set, 0)

acc:  0.95333331823349
--- embed 3.419264078140259 seconds ---
--- train 21.11810326576233 seconds ---
--- total 40.59562110900879 seconds ---
acc:  0.95333331823349
--- embed 3.4034903049468994 seconds ---
--- train 12.623661756515503 seconds ---
--- total 31.16509985923767 seconds ---
acc:  0.9549999833106995
--- embed 3.418227434158325 seconds ---
--- train 12.319038391113281 seconds ---
--- total 30.31583523750305 seconds ---
acc:  0.95333331823349
--- embed 3.3595666885375977 seconds ---
--- train 12.580832242965698 seconds ---
--- total 31.665188550949097 seconds ---
acc:  0.949999988079071
--- embed 3.4427661895751953 seconds ---
--- train 12.615651845932007 seconds ---
--- total 30.693434953689575 seconds ---
acc:  0.9516666531562805
--- embed 3.4687247276306152 seconds ---
--- train 12.653229475021362 seconds ---
--- total 31.406693696975708 seconds ---
acc:  0.9516666531562805
--- embed 3.4163429737091064 seconds ---
--- train 21.00416111946106 seconds ---
--- total 39.331142

In [ ]:
plot(results)

,Laplacian,DiagA,Correlation,Accuracy,Train_Time(s),Emb_Time(s),Total_Time(s)
set_01,True,True,True,0.952167,16.79191,3.41,35.29
set_02,True,True,False,0.480000,14.96333,3.42,33.36
set_03,True,False,True,0.951833,16.69279,3.46,36.04
set_04,True,False,False,0.480000,15.79760,3.44,35.08
set_05,False,True,True,0.953000,15.78440,3.32,24.09
set_06,False,True,False,0.954833,15.64148,3.31,23.85
set_07,False,False,True,0.953000,12.97473,3.45,21.31
set_08,False,False,False,0.954667,16.57532,3.43,24.88


## profiling


1.   https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.07-Timing-and-Profiling.ipynb
2.   https://perso.crans.org/besson/publis/notebooks/Profiling_in_a_Jupyter_notebook.html
3. have to disconnect the runtime then run for one task at a time

In [ ]:
! pip install memory_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=ad4dcf3435d162d5b00539976212730190f04121ed3f1d186df634201aa206f6
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


In [ ]:
%load_ext memory_profiler

In [ ]:
! pip install hypothesis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 387 kB 4.2 MB/s 


In [ ]:
# https://github.com/HypothesisWorks/hypothesis/issues/985
# doesn't help
import inspect
from hypothesis import settings

def is_debugging():
    for frame in inspect.stack():
        if frame[1].endswith("pydevd.py"):
            return True
    return False

if is_debugging():
    # The pycharm debugger requires to
    # run hypothesis with the `use_coverage=False` flag.
    try:
        settings.register_profile("debug", use_coverage=False)
        settings.load_profile(os.getenv(u'HYPOTHESIS_PROFILE', 'debug'))
    except:
        pass
else:
    settings.load_profile(os.getenv(u'HYPOTHESIS_PROFILE', 'default'))

### Sparse embed

In [ ]:
%%file mprun.py

import numpy as np
# for sparse matrix
from scipy import sparse

def graph_encoder_embed(X, Y, n, **kwargs):
  """
    graph embedding function
    input X is sparse csr matrix of adjacency matrix
    -- if there is a connection between node i and node j:
    ---- X(i,j) = 1, no edge weight
    ---- X(i,j) = edge weight.
    -- if there is no connection between node i and node j:
    ---- X(i,j) = 0, 
    ---- note there is no storage for this in sparse matrix. 
    ---- No storage means 0 in sparse matrix.
    input Y is a list of labels:
    -- value -1 indicate no lable
    -- value >=0 indicate real label
    input train_idx: a list of indices of input X for training set 
  """
  defaultKwargs = {'Correlation': True}
  kwargs = { **defaultKwargs, **kwargs}


  # assign k to the max along the first column
  # Note for python, label Y starts from 0. Python index starts from 0. thus size k should be max + 1
  k = Y[:,0].max() + 1

  #nk: 1*n array, contains the number of observations in each class
  nk = np.zeros((1,k))
  for i in range(k):
    nk[0,i] = np.count_nonzero(Y[:,0]==i)
  
  #W: sparse matrix for encoder marix. W[i,k] = {1/nk if Yi==k, otherwise 0}
  W = sparse.dok_matrix((n, k), dtype=np.float32)

  for i in range(Y.shape[0]):
    k_i = Y[i,0]
    if k_i >=0:
      W[i,k_i] = 1/nk[0,k_i]
  
  W = sparse.csr_matrix(W)
  Z = X.dot(W)
  
  return Z, W


Overwriting mprun.py


### origin embed

In [ ]:
def adj_to_edg(A):
  """
    input is the adjacency matrix: A
    other variables in this function:
    s: number of edges
    return edg_list -- matrix format with shape(edg_sum,3):
    example row in edg_list(matrix): [vertex1, vertex2, connection weight from Adj matrix]
  """
  # check the len of the second dimenson of A
  if A.shape[1] <= 3:
    edg = A
  else:
    n = A.shape[0]
    # construct the initial edgg_list matrix with the size of (edg_sum, 3)
    edg_list = []
    for i in range(n):
      for j in range(n):
        if A[i,j] > 0:
          edg_list.append([i, j, A[i,j]])
    edg = np.array(edg_list)
  return edg

In [ ]:
%%file mprun_origin.py

import numpy as np

def graph_encoder_embed(X,Y,n,**kwargs):
  """
    input X is s*3 edg list: nodei, nodej, connection weight(i,j)
    graph embedding function
  """
  defaultKwargs = {'Correlation': True}
  kwargs = { **defaultKwargs, **kwargs}
  
  # assign k to the max along the first column
  # Note for python, label Y starts from 0. Python index starts from 0. thus size k should be max + 1
  k = Y[:,0].max() + 1

  #nk: 1*n array, contains the number of observations in each class
  #W: encoder marix. W[i,k] = {1/nk if Yi==k, otherwise 0}
  nk = np.zeros((1,k))
  W = np.zeros((n,k))

  for i in range(k):
    nk[0,i] = np.count_nonzero(Y[:,0]==i)

  for i in range(Y.shape[0]):
    k_i = Y[i,0]
    if k_i >=0:
      W[i,k_i] = 1/nk[0,k_i]

  # Edge List Version in O(s)
  Z = np.zeros((n,k))
  i = 0
  for row in X:
    [v_i, v_j, edg_i_j] = row
    v_i = int(v_i)
    v_j = int(v_j)

    label_i = Y[v_i][0] 
    label_j = Y[v_j][0]

    if label_j >= 0:
      Z[v_i, label_j] = Z[v_i, label_j] + W[v_j, label_j]*edg_i_j
    if (label_i >= 0) and (v_i != v_j):
      Z[v_j, label_i] = Z[v_j, label_i] + W[v_i, label_i]*edg_i_j
  
  return Z, W

### cases

In [ ]:
import numpy as np
from scipy import sparse
import time

 #### case 10 with 5000 nodes

##### sparse

In [ ]:
n = 5000
case = Case(n)
case_10 = case.case_10_fully_known()
case_10.summary()

name:

    SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
5000
d:
<class 'int'>
3
X:
(5000, 5000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(5000, 1)
[[1]
 [0]
 [2]
 ...
 [2]
 [0]
 [1]]


In [ ]:
len(np.transpose(np.nonzero(case_10.X)))

2780918

In [ ]:
X_sparse = sparse.csr_matrix(case_10.X)

In [ ]:
from mprun import graph_encoder_embed

In [ ]:
%mprun -f graph_encoder_embed Z,W = graph_encoder_embed(X_sparse, case_10.Y, case_10.n)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 845, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 848, in disable
    sys.settrace(self._original_trace_function)



##### original

In [ ]:
n = 5000
case = Case(n)
case_10 = case.case_10_fully_known()
case_10.summary()

name:

    SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
5000
d:
<class 'int'>
3
X:
(5000, 5000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(5000, 1)
[[1]
 [0]
 [2]
 ...
 [2]
 [0]
 [1]]


In [ ]:
len(np.transpose(np.nonzero(case_10.X)))

2780918

In [ ]:
X_edg = adj_to_edg(case_10.X)

In [ ]:
from mprun_origin import graph_encoder_embed

In [ ]:
emb_strat = time.time()
Z, W = graph_encoder_embed(X_edg, case_10.Y, case_10.n)
emb_end = time.time()
emb_time = emb_end - emb_strat
print(emb_time)

10.877496480941772


doc: https://pypi.org/project/memory-profiler/
The first column represents the line number of the code that has been profiled, the second column (Mem usage) the memory usage of the Python interpreter after that line has been executed. The third column (Increment) represents the difference in memory of the current line with respect to the last one. The last column (Line Contents) prints the code that has been profiled.

In [ ]:
%mprun -f graph_encoder_embed Z,W = graph_encoder_embed(X_edg, case_10.Y, case_10.n)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 845, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 848, in disable
    sys.settrace(self._original_trace_function)



 #### case 10 with 3000 nodes

##### sparse

In [ ]:
n = 3000
case = Case(n)
case_10 = case.case_10_fully_known()
case_10.summary()

name:

    SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
Y:
(3000, 1)
[[1]
 [0]
 [2]
 ...
 [2]
 [2]
 [2]]


In [ ]:
len(np.transpose(np.nonzero(case_10.X)))

1000498

In [ ]:
X_sparse = sparse.csr_matrix(case_10.X)

In [ ]:
from mprun import graph_encoder_embed

In [ ]:
%mprun -f graph_encoder_embed Z,W = graph_encoder_embed(X_sparse, case_10.Y, case_10.n)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 845, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 848, in disable
    sys.settrace(self._original_trace_function)



##### original

In [ ]:
n = 3000
case = Case(n)
case_10 = case.case_10_fully_known()
case_10.summary()

name:

    SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
Y:
(3000, 1)
[[1]
 [0]
 [2]
 ...
 [2]
 [2]
 [2]]


In [ ]:
len(np.transpose(np.nonzero(case_10.X)))

1000498

In [ ]:
X_edg = adj_to_edg(case_10.X)

In [ ]:
from mprun_origin import graph_encoder_embed

In [ ]:
emb_strat = time.time()
Z, W = graph_encoder_embed(X_edg, case_10.Y, case_10.n)
emb_end = time.time()
emb_time = emb_end - emb_strat
print(emb_time)

3.5426878929138184


In [ ]:
%mprun -f graph_encoder_embed Z,W = graph_encoder_embed(X_edg, case_10.Y, case_10.n)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 845, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 848, in disable
    sys.settrace(self._original_trace_function)



#### case 20 with 5000 nodes

##### sparse

In [ ]:
n = 5000
case = Case(n)
case_20 = case.case_20_fully_known()
case_20.summary()

[[0.20530728]
 [0.25104189]
 [0.22868416]
 ...
 [0.06156643]
 [0.27507487]
 [0.12602176]]
name:

    DC-SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
5000
d:
<class 'int'>
3
X:
(5000, 5000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(5000, 1)
[[1]
 [0]
 [2]
 ...
 [2]
 [0]
 [1]]


In [ ]:
len(np.transpose(np.nonzero(case_20.X)))

184430

In [ ]:
X_sparse = sparse.csr_matrix(case_20.X)

In [ ]:
from mprun import graph_encoder_embed

In [ ]:
%mprun -f graph_encoder_embed Z,W = graph_encoder_embed(X_sparse, case_20.Y, case_20.n)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 845, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 848, in disable
    sys.settrace(self._original_trace_function)



##### original

In [ ]:
n = 5000
case = Case(n)
case_20 = case.case_20_fully_known()
case_20.summary()

[[0.20530728]
 [0.25104189]
 [0.22868416]
 ...
 [0.06156643]
 [0.27507487]
 [0.12602176]]
name:

    DC-SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
5000
d:
<class 'int'>
3
X:
(5000, 5000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(5000, 1)
[[1]
 [0]
 [2]
 ...
 [2]
 [0]
 [1]]


In [ ]:
len(np.transpose(np.nonzero(case_20.X)))

184430

In [ ]:
X_edg = adj_to_edg(case_20.X)

In [ ]:
from mprun_origin import graph_encoder_embed

In [ ]:
emb_strat = time.time()
Z, W = graph_encoder_embed(X_edg, case_20.Y, case_20.n)
emb_end = time.time()
emb_time = emb_end - emb_strat
print(emb_time)

0.6683666706085205


In [ ]:
%mprun -f graph_encoder_embed Z,W = graph_encoder_embed(X_edg, case_20.Y, case_20.n)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 845, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 848, in disable
    sys.settrace(self._original_trace_function)

